In [121]:
import pymysql
import pandas as pd
from tqdm import tqdm
from sqlalchemy import create_engine
import os
#import numpy as np


# database 조작
user_file = open("user.txt","r") # user 텍스트 파일 'r' 읽기 'w' 쓰기
user_file_read = user_file.readlines() # open한 텍스트 파일의 line 별로 가져오기
user_info = [] # info 빈 리스트 만들기

# 가져온 파일의 line의 공백 제거
for line in user_file_read :
    user_info.append(line.strip())
# 파일 닫기
user_file.close() 

# 데이터베이스 connect 생성
# pymysql.connect( host='아이피', user='유저계정', password='유저계정 패스워드', db='데이터베이스 명', charset='인코딩')
#db_connect = pymysql.connect(host='127.0.0.1', user=f'{user_info[0]}', password=f'{user_info[1]}', db='dbmaria', charset='utf8'))
db_connect = pymysql.connect(host='127.0.0.1', user='user', password=f'user1234', db='dbmaria', charset='utf8')
# connect cursor 생성
cursor = db_connect.cursor()

# 데이터 저장된 주소 지정
directory = './data/'
# 빈 리스트 생성
dfs = []

# 데이터파일 데이터프레임으로 변경
for filename in os.listdir(directory) :
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath, encoding='cp949')
        
        # 파일 명에서 구, 년월 값 가져와서 삽입('GU', 'YM')
        df['GU'] = filename.split('_')[1]
        df['YM'] = filename.split('_')[2].split('.')[0]
        
        # 각 파일별 데이터프레임 dfs 리스트에 저장
        dfs.append(df)

# dfs 리스트에 저장된 데이터프레임들 concat으로 합쳐서 하나의 데이터프레임 생성
result = pd.concat(dfs, ignore_index=True)

# 데이터프레임 df로 재 선언
df = result

# 데이터프레임의 'YM' 열에 데이터를 str 형식으로 변경
df['YM'] = df['YM'].astype(str)

# 데이터프레임의 NaN값 ""로 변환
df = df.fillna("") 

# 데이터프레임 중복값 제거
df = df.drop_duplicates() 

# 데이터프레임 중복값 찾기
df[df.duplicated(keep=False)] 

# 데이터베이스 테이블 생성
# cursor.execute('query')
cursor.execute("""CREATE TABLE IF NOT EXISTS gu_real_estate (
     transaction_id INT AUTO_INCREMENT,
     transaction_amount VARCHAR(255),
     transaction_type VARCHAR(255),
     construction_year VARCHAR(255),
     year INT,
     dong VARCHAR(255),
     registration_date VARCHAR(255),
     seller VARCHAR(255),
     buyer VARCHAR(255),
     legal_dong VARCHAR(255),
     apartment_name VARCHAR(255),
     month INT,
     day INT,
     exclusive_area FLOAT,
     broker_location VARCHAR(255),
     jibun VARCHAR(255),
     region_code INT,
     floor INT,
     cancellation_reason_date VARCHAR(255),
     cancellation_yn VARCHAR(255),
     GU_id VARCHAR(255),
     YM_id VARCHAR(255),
     PRIMARY KEY (transaction_id, GU_id, YM_id))""") 

# # 데이터 프레임의 데이터를 데이터베이스에 삽입
for index, row in df.iterrows():
    cursor.execute("""
        INSERT INTO gu_real_estate (
            transaction_amount,
            transaction_type,
            construction_year,
            year,
            dong,
            registration_date,
            seller,
            buyer,
            legal_dong,
            apartment_name,
            month,
            day,
            exclusive_area,
            broker_location,
            jibun,
            region_code,
            floor,
            cancellation_reason_date,
            cancellation_yn,
            GU_id,
            YM_id
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['거래금액'],
        row['거래유형'],
        row['건축년도'],
        row['년'],
        row['동'],
        row['등기일자'],
        row['매도자'],
        row['매수자'],
        row['법정동'],
        row['아파트'],
        row['월'],
        row['일'],
        row['전용면적'],
        row['중개사소재지'],
        row['지번'],
        row['지역코드'],
        row['층'],
        row['해제사유발생일'],
        row['해제여부'],
        row['GU'],
        row['YM']
    ))

# db에 커밋
db_connect.commit() 

# db 연결 끊기
db_connect.close() 

In [103]:
#for filename in os.listdir(directory)[:1] :
#    if filename.endswith(".csv"):
#        filepath = os.path.join(directory, filename)
#        df = pd.read_csv(filepath, encoding='cp949')
#        df['GU'] = filename.split('_')[1]
#        df['YM'] = filename.split('_')[2].split('.')[0]


In [82]:
# 법정동 자료
lawd_cd_f = []
lawd_cd_con = []
code_list = []
name_list = []
lawd_cd = pd.read_csv('data/법정동코드 전체자료.txt', sep='\t', encoding='cp949')
lawd_cd.columns = ['code', 'name', 'check']
lawd_cd = lawd_cd.drop(columns='check')

for i in lawd_cd.index :
    lawd_cd_f.append(lawd_cd['code'][i].astype(str)[0:5])
    
lawd_cd['code'] = lawd_cd_f
for i in lawd_cd.index :
    a = lawd_cd['name'][i].split()
    if len(a) == 1 : 
        continue
    elif len(a) == 3 :
        continue
    if a[0] != '서울특별시' :
        continue
    code_list.append(lawd_cd['code'][i])
    name_list.append(a[1])
    
lawd_cd_con.append(code_list)
lawd_cd_con.append(name_list)
lawd_df = pd.DataFrame(lawd_cd_con).T
lawd_df.columns = ['code', 'name']

       name
code       
11110   종로구
11140    중구
11170   용산구
11200   성동구
11215   광진구
11230  동대문구
11260   중랑구
11290   성북구
11305   강북구
11320   도봉구
11350   노원구
11380   은평구
11410  서대문구
11440   마포구
11470   양천구
11500   강서구
11530   구로구
11545   금천구
11560  영등포구
11590   동작구
11620   관악구
11650   서초구
11680   강남구
11710   송파구
11740   강동구


In [ ]:
for i in df_test.columns :
    if i == '년' :
        continue
    elif i == '월' :
        continue
    elif i == '지역코드' :
        continue
    else :
        df_test.drop(columns=i,inplace=True)
        
df_test